In [1]:
import torch

In [8]:
a = torch.ones(3)

In [19]:
torch.transpose(a, 0, 0)

TypeError: transpose() received an invalid combination of arguments - got (builtin_function_or_method, int, int), but expected one of:
 * (Tensor input, int dim0, int dim1)
      didn't match because some of the arguments have invalid types: (!builtin_function_or_method!, int, int)
 * (Tensor input, name dim0, name dim1)
      didn't match because some of the arguments have invalid types: (!builtin_function_or_method!, !int!, !int!)


In [15]:
a

<function Tensor.transpose>

In [10]:
a.shape

torch.Size([3])

In [21]:
img_t = torch.randn(3, 5, 5)
weights = torch.tensor([[0.2126, 0.7152, 0.0722]])

In [22]:
img_t.shape

torch.Size([3, 5, 5])

In [24]:
img_t

tensor([[[ 1.7341, -1.0806,  0.6889, -0.4402, -0.7036],
         [ 1.2944, -1.2241, -1.0547,  2.3595, -0.2258],
         [-0.1752, -1.0398, -0.3001, -0.1257,  0.9464],
         [-0.0722,  1.5444,  1.2444,  0.7804, -1.0533],
         [-1.8047, -0.4202,  0.3887,  0.1772,  0.7409]],

        [[-0.2531, -2.1769,  0.9891,  0.8389, -0.4451],
         [ 0.6076, -2.5692, -0.0933, -0.2606,  1.2190],
         [ 0.5746, -1.6864,  0.8541, -0.7754, -0.4961],
         [ 0.3599, -0.4335, -0.8770,  0.0801, -0.0179],
         [-0.1283,  0.5582, -0.3465, -0.5952,  1.0396]],

        [[ 0.7291,  0.1034, -0.2362, -0.3158,  0.6135],
         [ 0.5945,  0.5252, -2.2485,  0.7474,  1.1713],
         [-1.5469,  0.1246, -0.7611,  0.6384, -0.2101],
         [-1.0749, -0.6130,  0.4406, -1.2493,  1.4020],
         [-0.2210,  0.8272, -1.0042, -0.2435, -0.6698]]])

In [23]:
img_t[0, 1, 1]

tensor(-1.2241)

In [25]:
batch_t = torch.randn(2, 3, 5, 5)

In [26]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [27]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([1, 3, 1, 1]))

In [29]:
img_t.shape

torch.Size([3, 5, 5])

In [28]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

RuntimeError: einsum(): the number of subscripts in the equation (1) does not match the number of dimensions (2) for operand 1 and no ellipsis was given

In [30]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])

C:\Users\MLDL\AppData\Local\Temp\ipykernel_23232\1901091524.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10/core/TensorImpl.h:1761.)
  weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])


In [31]:
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [32]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')

In [33]:
print("img named: ", img_named.shape, img_named.names)
print("batch named: ", batch_named.shape, batch_named.names)

img named:  torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named:  torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [34]:
weights_aligned = weights_named.align_as(img_named)

In [35]:
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [36]:
gray_named = (img_named * weights_aligned).sum('channels')

In [37]:
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [38]:
gray_named = (img_named[..., :3] * weights_named).sum('channels')

RuntimeError: Error when attempting to broadcast dims ['channels', 'rows', 'columns'] and dims ['channels']: dim 'columns' and dim 'channels' are at the same position from the right but do not match.

In [39]:
gray_plain = gray_named.rename(None)

In [40]:
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))